# LightFM, фактаризационные матрицы, неявные отклики

Мы уже видели, построение модели на примере коллаборативной фильтрации. Для оценок того, что пользователю понравился фильм использовался рейтинг поставленный пользователем. Однако это не всегда правильный подход использовать явные отклики пользователя. Обучая моедль и пытаясь получить максимально близкие рейтинги к фильмам, которые пользователь уже посмотрел, мы упускаем факт того какие фильмы пользоваль не посмотрел. Факт того, что пользователь не посмотрел и не поставил рейтинг, значит что пользователь не выбрал их в первую очередь. Что показывает осознанный выбор пользователя и может быть использовано в качестве негативного отклика и таким образом обучить модель на неявных откликах

В данном примере мы будем использовать в качестве target поставленный рейтинг и не просмотр фильма. В качестве модели будем использовать библиотеку основанную на идее матричных разложений LightFM <br>
Подробнее о матричных разложениях: https://developers.google.com/machine-learning/recommendation/collaborative/matrix
Библиотека LightFM: https://making.lyst.com/lightfm/docs/home.html

In [9]:
# !pip install lightfm

In [3]:
import numpy as np

from lightfm.datasets import fetch_movielens

movielens = fetch_movielens()

In [4]:
for key, value in movielens.items():
    print(key, type(value), value.shape)

train <class 'scipy.sparse.coo.coo_matrix'> (943, 1682)
test <class 'scipy.sparse.coo.coo_matrix'> (943, 1682)
item_features <class 'scipy.sparse.csr.csr_matrix'> (1682, 1682)
item_feature_labels <class 'numpy.ndarray'> (1682,)
item_labels <class 'numpy.ndarray'> (1682,)


Обратите внимание что входными данными будут **sparse matix** или разряженные матрицы. Sparce matrix используются более экономного хранения данных, в которых нулевых элементов значительно больше ненулевых

In [28]:
y = np.bincount(train.toarray()[0].astype('int64'))
ii = np.nonzero(y)[0]
list(zip(ii,y[ii]))

[(0, 1420), (1, 25), (2, 27), (3, 54), (4, 77), (5, 79)]

In [6]:
train = movielens['train']
test = movielens['test']

In [33]:
from lightfm import LightFM
from lightfm.evaluation import precision_at_k, recall_at_k
from lightfm.evaluation import auc_score

model = LightFM(learning_rate=0.05, loss='bpr')
model.fit(train, epochs=10)

train_precision_at_10 = precision_at_k(model, train, k=10).mean()
test_precision_at_10 = precision_at_k(model, test, k=10, train_interactions=train).mean()

train_recall_at_10 = recall_at_k(model, train, k=10).mean()
test_recall_at_10 = recall_at_k(model, test, k=10).mean()

train_auc = auc_score(model, train).mean()
test_auc = auc_score(model, test, train_interactions=train).mean()

print('Precision@10: train %.2f, test %.2f.' % (train_precision_at_10, test_precision_at_10))
print('Recall@10: train %.2f, test %.2f.' % (train_recall_at_10, test_recall_at_10))
print('AUC: train %.2f, test %.2f.' % (train_auc, test_auc))

Precision@10: train 0.58, test 0.19.
Recall@10: train 0.11, test 0.10.
AUC: train 0.89, test 0.88.


In [8]:
model = LightFM(learning_rate=0.05, loss='warp')

model.fit_partial(train, epochs=10)

train_precision = precision_at_k(model, train, k=10).mean()
test_precision = precision_at_k(model, test, k=10, train_interactions=train).mean()

train_auc = auc_score(model, train).mean()
test_auc = auc_score(model, test, train_interactions=train).mean()

print('Precision: train %.2f, test %.2f.' % (train_precision, test_precision))
print('AUC: train %.2f, test %.2f.' % (train_auc, test_auc))

Precision: train 0.61, test 0.22.
AUC: train 0.94, test 0.93.
